In [1]:
library(gmodels)

In [2]:
data(infert, package = "datasets")

In [3]:
head(infert)

education,age,parity,induced,case,spontaneous,stratum,pooled.stratum
0-5yrs,26,6,1,1,2,1,3
0-5yrs,42,1,1,1,0,2,1
0-5yrs,39,6,2,1,0,3,4
0-5yrs,34,4,2,1,0,4,2
6-11yrs,35,3,1,1,1,5,32
6-11yrs,36,4,2,1,1,6,36


In [4]:
CrossTable(infert$education, infert$induced, prop.t=TRUE, prop.r=TRUE, prop.c=TRUE, chisq=TRUE)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  248 

 
                 | infert$induced 
infert$education |         0 |         1 |         2 | Row Total | 
-----------------|-----------|-----------|-----------|-----------|
          0-5yrs |         4 |         2 |         6 |        12 | 
                 |     1.232 |     0.506 |     9.898 |           | 
                 |     0.333 |     0.167 |     0.500 |     0.048 | 
                 |     0.028 |     0.029 |     0.162 |           | 
                 |     0.016 |     0.008 |     0.024 |           | 
-----------------|-----------|-----------|-----------|-----------|
         6-11yrs |        78 |        27 |        15 |       120 | 
                 |     1.121 |     1.059 |     0.471 |           | 
                 |     0.650 |

In [5]:
N = matrix(c(5, 18, 19, 12, 3, 7, 46, 29, 40), 
 nrow = 3,
 
 dimnames = list(
 "Sinistro" = c('Explosão', 'Incêndio', 'Colapso estrutural'),
 "Categoria" = c('Nível 1', 'Nível 2', 'Nível 3')))  
N

,Nível 1,Nível 2,Nível 3
Explosão,5,12,46
Incêndio,18,3,29
Colapso estrutural,19,7,40


In [6]:
# Percentuais globais
n <- sum(N)
P <- N/n
P

,Nível 1,Nível 2,Nível 3
Explosão,0.02793296,0.06703911,0.2569832
Incêndio,0.10055866,0.01675978,0.1620112
Colapso estrutural,0.10614525,0.03910615,0.2234637


In [7]:
# Totais
colunas.totais <- colSums(P)
colunas.totais

linhas.totais <- rowSums(P)
linhas.totais

Nível 1   Nível 2   Nível 3 
0.2346369 0.1229050 0.6424581

Explosão           Incêndio Colapso estrutural 
         0.3519553          0.2793296          0.3687151

In [8]:
# Probabilidades esperadas
E <- linhas.totais %o% colunas.totais
E

,Nível 1,Nível 2,Nível 3
Explosão,0.08258169,0.04325708,0.2261165
Incêndio,0.06554103,0.03433101,0.1794576
Colapso estrutural,0.08651415,0.04531694,0.2368840


In [9]:
# Resíduos
R <- P - E
R

,Nível 1,Nível 2,Nível 3
Explosão,-0.05464873,0.023782029,0.03086670
Incêndio,0.03501763,-0.017571237,-0.01744640
Colapso estrutural,0.01963110,-0.006210792,-0.01342031


In [10]:
# Índices
I <- R/E
I

,Nível 1,Nível 2,Nível 3
Explosão,-0.6617536,0.5497835,0.13650794
Incêndio,0.5342857,-0.5118182,-0.09721739
Colapso estrutural,0.2269120,-0.1370523,-0.05665349


In [11]:
# Padronização dos dados
Z = I * sqrt(E)
SVD = svd(Z) # Singular Value Decomposition
rownames(SVD$u) <- rownames(P) # Vetor singular a esquerda
rownames(SVD$v) <- colnames(P) # Vetor singular a direita
SVD
autovalores <- SVD$d^2
prop.table(autovalores)

Explosão,-0.7764404,-0.2125676,0.5932582
Incêndio,0.5748453,-0.6246785,0.5285164
Colapso estrutural,0.2582502,0.7513932,0.6072191
Nível 1,0.8179138,0.3104515,0.4843933
Nível 2,-0.5066013,0.7876866,0.3505781
Nível 3,-0.2727126,-0.5321370,0.8015348


[1] 9.969312e-01 3.068826e-03 2.388213e-32

In [12]:
coordenadas.padronizadas.colunas = sweep(SVD$v, 1, sqrt(colunas.totais), "/")
coordenadas.padronizadas.colunas

coordenadas.padronizadas.linhas = sweep(SVD$u, 1, sqrt(linhas.totais), "/")
coordenadas.padronizadas.linhas

Nível 1,1.688532,0.6409079,1
Nível 2,-1.445045,2.2468217,1
Nível 3,-0.340238,-0.6638975,1


Explosão,-1.3087731,-0.3583054,1
Incêndio,1.0876584,-1.1819472,1
Colapso estrutural,0.4252998,1.2374333,1


In [13]:
# Coordenadas principais
coordenadas.principais.colunas = sweep(coordenadas.padronizadas.colunas, 2, SVD$d, "*")
coordenadas.principais.colunas

coordenadas.principais.linhas = sweep(coordenadas.padronizadas.linhas, 2, SVD$d, "*")
coordenadas.principais.linhas

Nível 1,0.5027305,0.01058705,4.608184e-17
Nível 2,-0.4302365,0.03711488,4.608184e-17
Nível 3,-0.1012998,-0.01096681,4.608184e-17


Explosão,-0.3896639,-0.005918787,4.608184e-17
Incêndio,0.3238309,-0.019524390,4.608184e-17
Colapso estrutural,0.1266255,0.020440956,4.608184e-17


In [14]:
# Medidas de qualidade
pc = rbind(coordenadas.principais.linhas, coordenadas.principais.colunas) 
prop.table(pc ^ 2, 1)

Explosão,0.9997693,0.0002306666,1.398231e-32
Incêndio,0.9963781,0.0036219499,2.017656e-32
Colapso estrutural,0.9746027,0.0253973202,1.290759e-31
Nível 1,0.9995567,0.0004432890,8.398399e-33
Nível 2,0.9926131,0.0073868870,1.138741e-32
Nível 3,0.9884153,0.0115846531,2.045417e-31


In [15]:
library(ca)
mytable <- N

prop.table(N, 1) # Linhas
prop.table(N, 2) # Colunas

fit <- ca(N)

print(fit) # Resultados básic results 
summary(fit) # extended results 

plot(fit) # symmetric map
plot(fit, mass = TRUE, contrib = "absolute", map =
  	"rowgreen", arrows = c(FALSE, TRUE)) # asymmetric map

,Nível 1,Nível 2,Nível 3
Explosão,0.07936508,0.1904762,0.7301587
Incêndio,0.36000000,0.0600000,0.5800000
Colapso estrutural,0.28787879,0.1060606,0.6060606


,Nível 1,Nível 2,Nível 3
Explosão,0.1190476,0.5454545,0.4000000
Incêndio,0.4285714,0.1363636,0.2521739
Colapso estrutural,0.4523810,0.3181818,0.3478261



 Principal inertias (eigenvalues):
           1        2       
Value      0.088644 0.000273
Percentage 99.69%   0.31%   


 Rows:
         Explosão  Incêndio Colapso estrutural
Mass     0.351955  0.279330           0.368715
ChiDist  0.389709  0.324419           0.128265
Inertia  0.053453  0.029399           0.006066
Dim. 1  -1.308773  1.087658           0.425300
Dim. 2  -0.358305 -1.181947           1.237433


 Columns:
         Nível 1   Nível 2   Nível 3
Mass    0.234637  0.122905  0.642458
ChiDist 0.502842  0.431834  0.101892
Inertia 0.059328  0.022919  0.006670
Dim. 1  1.688532 -1.445045 -0.340238
Dim. 2  0.640908  2.246822 -0.663897


Warning message in abbreviate(rnames.temp, 4):
“abbreviate used with non-ASCII chars”Warning message in abbreviate(cnames.temp, 4):
“abbreviate used with non-ASCII chars”


Principal inertias (eigenvalues):

 dim    value      %   cum%   scree plot               
 1      0.088644  99.7  99.7  *************************
 2      0.000273   0.3 100.0                           
        -------- -----                                 
 Total: 0.088917 100.0                                 


Rows:
    name   mass  qlt  inr    k=1  cor ctr    k=2 cor ctr  
1 | Expl |  352 1000  601 | -390 1000 603 |   -6   0  45 |
2 | Incn |  279 1000  331 |  324  996 330 |  -20   4 390 |
3 | Clps |  369 1000   68 |  127  975  67 |   20  25 565 |

Columns:
    name   mass  qlt  inr    k=1  cor ctr    k=2 cor ctr  
1 | Nvl1 |  235 1000  667 |  503 1000 669 |   11   0  96 |
2 | Nvl2 |  123 1000  258 | -430  993 257 |   37   7 620 |
3 | Nvl3 |  642 1000   75 | -101  988  74 |  -11  12 283 |